In [1]:
import re     
import nltk  
from nltk.corpus import stopwords
import string 
import warnings 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt  
import xgboost as xgb
from sklearn import tree
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

In [2]:
xtrain = pd.read_csv('train.csv')

xtrain

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [4]:
xtrain['text'] = xtrain['text'].apply(lambda x: clean_text(x))

In [5]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
xtrain['text'] = xtrain['text'].apply(lambda x: tokenizer.tokenize(x))

In [6]:
def remove_stopwords(text):
    words = []
    for w in text:
        if w not in stopwords.words('english'):
            words.append(w)
    #words = [w for w in text if w not in stopwords.words('english')]
    return words

xtrain['text'] = xtrain['text'].apply(lambda x: remove_stopwords(x))

In [7]:
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

In [8]:
xtrain['text'] = xtrain['text'].apply(lambda x: combine_text(x))

In [9]:
tokenized_tweet = xtrain.text.apply(lambda x: x.split())
tokenized_tweet[1]

['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada']

In [10]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_glovec[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [11]:
model_glovec = {}
with open("glove.twitter.27B.200d.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        model_glovec[word] = vector

In [12]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(7613, 200)

SyntaxError: positional argument follows keyword argument (<ipython-input-13-8fd122764ab1>, line 1)

In [16]:
xtest = pd.read_csv('test.csv')
xtest.text = xtest.text.apply(lambda x: clean_text(x))
xtest.text = xtest.text.apply(lambda x: tokenizer.tokenize(x))
xtest.text = xtest.text.apply(lambda x: remove_stopwords(x))
xtest.text = xtest.text.apply(lambda x: combine_text(x))
tokenized_test = xtest.text.apply(lambda x: x.split()) # tokenizing 
wordvec_test = np.zeros((len(tokenized_test), 200)) 
for i in range(len(tokenized_test)):
    wordvec_test[i,:] = word_vector(tokenized_test[i], 200)
wordvect_df = pd.DataFrame(wordvec_test)
wordvect_df.shape

(3263, 200)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [23]:
xtrain_gv = wordvec_df
xtest_gv = wordvect_df
ytrain= xtrain.target


In [19]:
data_dmatrix = xgb.DMatrix(data=xtrain_gv,label=ytrain)

In [24]:
ytrain

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [25]:
alg = xgb.XGBClassifier(learning_rate=0.1, 
                        n_estimators=1000,
                        max_depth=15, 
                        min_child_weight=3,
                        gamma=0.2, 
                        subsample=0.6,
                        objective='binary:logistic',
                        nthread=4)

xgb_param = alg.get_xgb_params()
xgtrain = xgb.DMatrix(xtrain_gv, label=ytrain)
xgtest = xgb.DMatrix(xtest_gv)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=5,
                  early_stopping_rounds=40)
alg.set_params(n_estimators=cvresult.shape[0])
alg.fit(xtrain_gv, ytrain, eval_metric='auc')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=85, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.6,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [26]:
A=alg.predict(xtest_gv)

In [ ]:
ids=[]
with open("sample_submission.csv", 'r') as f:
    for line in f:
        vals = line.split(',')
        if vals[0]!='id':
            ids.append(int(vals[0]))

In [21]:
en = open("submission.csv", 'w')
n=0
en.write("id,target\n")
for i in ids:
    en.write(str(i) +','+ str(int((A[n])))+'\n')    
    n+=1
en.close()

NameError: name 'ids' is not defined